# Genotyping-by-sequencing

## Introduction

This workshop focuses on large SNP data sets such as those obtained from genotyping-by-sequencing (GBS) for population genetic analysis in R. GBS is one of several techniques used to genotype populations using high throughput sequencing (HTS). In GBS, the genome is reduced in representation by using restriction enzymes, and then sequencing these products using HTS.

We will use a data set of 124 samples of the Lumpfish  (_Cyclopterus lumpus_). They are found in the cold waters of the Arctic, North Atlantic, and North Pacific oceans. There are used as cleaner fish by the Salmon farming industry.

Populations were obtained from three locations to investigate population structure, genetic diversity and stock sustainability in order to establish a breeding programme. A total of 124 samples were sequenced using the Illumina HiSeq 2500 technology with 150 bp paired-end reads and a target insert size of 600 bp. Currently, there is little information about the population structure of _C. lumpus_. The VCF data for this population can be downloaded from: [fish.snps.vcf.gz](https://github.com/pseudogene/aqupgp3-workshops/raw/master/fish.snps.vcf.gz).

To obtain variant calls in form of VCF data, the FASTQ reads from HTS were mapped to the reference genome of _C. lumpus_ using bowtie2 (Langmead & Salzberg, 2012). Variants were called using the GATK HaplotypeCaller (McKenna et al., 2010). Data was filtered as follows:

 * A minimum read-depth of 5x.
 * Only variants with a QC greater than 20 were retained.
 * Variants with more than 30% missing data were removed.

In addition to the VCF data, we have included the file `fish.pop.tsv`, a tab-delimited text file that includes the name of the sample, country of origin, and the population from where it was sampled. The file is available for download at: [fish.pop.tsv](https://github.com/pseudogene/aqupgp3-workshops/raw/master/fish.pop.tsv). This link will likely open the data in a browser. Save the data onto your computer as an ASCII text file with the same name.

## Opening and examining the vcf file

Let's first load the libraries needed for analysis:

In [ ]:
library(vcfR)
library(ape)
library(poppr)
library(RColorBrewer)
library(tidyr)
library(ggplot2)

options(repr.plot.width=16, repr.plot.height=8)

Make sure you are in the right folder with the downloaded files available. Some of these packages will print a message when they are loaded. Here we suppressed this information. When you load these packages, you may see more output than presented here.

Next, let’s open the VCF file using vcfR and check that we have 124 samples and 2610 SNPs:

In [ ]:
fish_data <- read.vcfR("fish.snps.vcf.gz")

Once we’ve loaded the data into R, we can validate it by entering our object’s name in the console:

In [ ]:
fish_data

VCF data does not typically include any sort of population information. We have to load this data separately from the text-delimited file we downloaded above that includes the ID of samples and the state where the samples were obtained from. Let’s load the `fish.pop.tsv` file into memory by using the `read.table()` function:

In [ ]:
fish_pop <- read.delim("fish.pop.tsv", header=TRUE,  sep="\t")

We can now check that all the samples in the population table.

In [ ]:
head(fish_pop)

In [ ]:
table(fish_pop$origin)
barplot(table(fish_pop$origin), col=funky(17), las=3, ylab="Sample size");

## Converting the dataset to a genlight object

The next step is to convert the data set into an object that is usable by `poppr`, `adegenet`, or any of the other population genetics packages in R. The `vcfR` package contains multiple functions to convert data into other formats (see the `converting_data` vignette of _vcfR_): `vignette("converting_data")`. For our particular purpose we want to convert the _vcfR_ object into a `genlight` object. We can use the `vcfR2genlight` function for this:

In [ ]:
fish_genotypes <- vcfR2genlight(fish_data)

we are required to specify the ploidy of the organism in order to calculate some population genetic metrics. _C. lumpus_ is a diploid organism, so we will specify a ploidy of two. All `genlight` objects have ploidy slots, in which the user can specify the ploidy of each individual sample, or once for the entire population. We can assume that every sample of _C. lumpus_ is diploid and we will specify a ploidy of 2 for the entire population. Note that while a `genlight` object can support individuals of different ploidy, within each individual all loci must be of the same ploidy.

In [ ]:
ploidy(fish_genotypes) <- 2

Our biological question requires predetermined populations. We can add them to the genlight object as part of the `pop` (population) slot. In order to specify the population, we added the `origin` column from our `fish_pop` data frame to the pop slot of our `genlight` object:

In [ ]:
pop(fish_genotypes) <- fish_pop$origin

We now end up with a `genlight` object of filtered VCF data:

In [ ]:
fish_genotypes

In [ ]:
as.matrix(fish_genotypes)

Next, let's get started with our first analyses.

## Population genetic analyses for GBS data
### Distance matrices

Let's create a pairwise genetic distance matrix for individuals or populations (i.e., groups of individuals).

To summarise, we can create a distance matrix from a `genlight` object using `dist()`:

In [ ]:
dist(fish_genotypes)

Note, that we have not specified what the variable x is. We can find documentation for this function with `?dist`.

There are also functions to create distance matrices from genlight objects that exist in other packages. The function `bitwise.dist()` in the package _poppr_ is an example. We can find documentation for this function with `?poppr::bitwise.dist`. Again, you need to know where to look for this information or you may not find it. We can use this function as follows.

In [ ]:
poppr::bitwise.dist(fish_genotypes)

### Distance tree

Let's start our analysis by building a genetic distance tree that represents the genetic relatedness of the samples. The similarity between samples and groups of samples is represented by the branch length. In most trees, the branch length is represented by the number of substitutions per site for a cluster or a sample. When samples are very similar, they are grouped by short branches. The longer the branch, the higher the number of substitutions and the higher the genetic distance is between samples or clusters.

For this tutorial, we will build a distance tree to obtain an initial assessment of the population structure of the lumpfish samples. We will reconstruct a distance tree based on the UPGMA algorithm, with 100 bootstrap replicates to assess branch support:

In [ ]:
tree <- aboot(fish_genotypes, tree = "upgma", distance = bitwise.dist, sample = 100, showtree = T, cutoff = 50, quiet = T)

Next, we will colour the tips of the tree based on the population of origin of the samples, and draw conclusions from what we observe in the tree:

In [ ]:
cols <- brewer.pal(n = nPop(fish_genotypes), name = "Dark2")
plot.phylo(tree, cex = 0.8, font = 2, adj = 0, tip.color =  cols[pop(fish_genotypes)])
nodelabels(tree$node.label, adj = c(1.3, -0.5), frame = "n", cex = 0.8,font = 3, xpd = TRUE)
legend('topleft', legend = unique(fish_pop$origin), fill = cols, border = FALSE, bty = "n", cex = 2)
axis(side = 1)
title(xlab = "Genetic distance (proportion of loci that are different)")

We observe that samples do not cluster exclusively by region. Instead, we observe a cluster with mainly Outer Hebrides samples (purple), but also containing a few Norway (red) and Iceland (green) samples. The second, lower cluster contains predominantly samples from Norway (red) and Iceland (green).

### Principal components analysis

A principal components analysis (PCA) converts the observed SNP data into a set of values of linearly uncorrelated variables called principal components that summarise the variation between samples. We can perform a PCA on our genlight object by using the `glPCA` function.

In [ ]:
fish.pca <- glPca(fish_genotypes, nf = 3)

In [ ]:
head(100*fish.pca$eig/sum(fish.pca$eig), n=10)

barplot(100*fish.pca$eig/sum(fish.pca$eig), col = heat.colors(50), main="PCA Eigenvalues")
title(ylab="Percent of variance\nexplained", line = 2)
title(xlab="Eigenvalues", line = 1)

The bar plot shows that we can retain the first 3 PCAs, which cumulatively explain only 8.4 percent of the variance of the data.

To view the results of the PCA we can use the package `ggplot2`. We need to convert the data frame that contains the principal components (`fish.pca$scores`) into the new object `fish.pca.scores`. In addition, we will add the population values as a new column in our rubi.pca.scores object, in order to be able to colour samples by population.

`ggplot2` will plot the PCA, colour the samples by population, and create ellipses that include 95\% of the data for each the population:

In [ ]:
fish.pca.scores <- as.data.frame(fish.pca$scores)
fish.pca.scores$pop <- pop(fish_genotypes)

set.seed(9)
p <- ggplot(fish.pca.scores, aes(x=PC1, y=PC2, colour=pop)) 
p <- p + geom_point(size=2)
p <- p + stat_ellipse(level = 0.95, size = 1)
p <- p + scale_color_manual(values = cols) 
p <- p + geom_hline(yintercept = 0) 
p <- p + geom_vline(xintercept = 0) 
p <- p + theme_bw()

p

The PCA produces a pattern similar our previous results. We observe that PC1 distinguishes samples from Outer Hebrides and Iceland. The Norway samples form a tight cluster with a narrow ellipse in red.

We can further explore population assignments using a discriminant analysis of principal components (DAPC).

### DAPC

The DAPC is a multivariate statistical approach that uses populations defined a priori to maximise the variance among populations in the sample by partitioning it into between-population and within-population components. DAPC thus maximises the discrimination between groups.

Often, we want to infer population structure by determining the number of clusters (groups) observed without prior knowledge. Several approaches can be used to infer groups such as for example K-means clustering, Bayesian clustering using STRUCTURE, and multivariate methods such as Discriminant Analysis of Principal Components (DAPC) (Pritchard, Stephens & Donnelly, 2000; Jombart, Devillard & Balloux, 2010; Grünwald & Goss, 2011). A STRUCTURE-like approach assumes that markers are not linked and that populations are panmictic (Pritchard et al., 2000). To use model-free methods K-means clustering based on genetic distance or DAPC are more convenient approaches for populations that are clonal or partially clonal.

DAPC requires a genlight object with populations defined _a priori_. We already have this `genlight` object from the above steps. Usually, we use the number of principal components and discriminant axes that maximise the variance between populations; but our objective here is to calculate the population assignments based on the results of the PCA. We will use the same parameters as in the PCA to make the results comparable between both methods. These parameters (`n.pca=3` and `n.da=2`) will be used to reconstruct the DAPC, obtain the assignment of the samples to each population, and suggest admixture between geographical locations.

In [ ]:
pnw.dapc <- dapc(fish_genotypes, n.pca = 3, n.da = 2)

To confirm that the DAPC is similar to the PCA we can plot the data in a scatter plot.

In [ ]:
scatter(pnw.dapc, col = cols, cex = 2, legend = TRUE, clabel = F, posi.leg = "bottomleft", scree.pca = FALSE,
        posi.pca = "topleft", cleg = 0.75)

We see that the results of the PCA and DAPC are very similar. The DAPC object we created includes the population membership probability for each sample to each of the predetermined populations. To visualize the posterior assignments of each sample, we use a composite stacked bar plot (compoplot). A compoplot illustrates the probability of population membership on the y-axis. Each sample is a bin on the x-axis, and the assigned probability of population membership is shown as a stacked bar chart with clusters or populations shown in colour.

In [ ]:
#compoplot(pnw.dapc,col = function(x) cols, posi = 'top')
compoplot(pnw.dapc,col = cols, posi = 'top')

These plots are hard to interpret, and we will thus separate the samples by population.
`ggplot2` can be used to reconstruct these plots, but we need to convert the data into a `ggplot2` friendly object. We will extract the DAPC calculated population membership assignments (`pnw.dapc$posterior`) into a new data frame (`dapc.results`), include the original population assignment as a new column in the data frame (`dapc.results$pop`), and add a column that includes the sample names (`dapc.results$indNames`).

In [ ]:
dapc.results <- as.data.frame(pnw.dapc$posterior)
dapc.results$pop <- pop(fish_genotypes)
dapc.results$indNames <- rownames(dapc.results)

`ggplot2` has specific requirements for the structure of the data frame format, as it requires each observation in rows, and all different values of these observations in columns (i.e., a long format data frame). To transform the data frame, we use `pivot_longer` from the package `tidyr`.

In [ ]:
dapc.results <- pivot_longer(dapc.results, -c(pop, indNames))

We have now reorganized the data frame into the required format, where each membership probability observation for a given population is a row with the sample name, original population, and assigned population as columns.

In [ ]:
head(dapc.results)

Then, we rename the columns into more familiar terms:

In [ ]:
colnames(dapc.results) <- c("Original_Pop","Sample","Assigned_Pop","Posterior_membership_probability")
head(dapc.results)

`ggplot2` will plot the dapc.results data frame we reorganized using pivot_longer, using the samples on the X-axis and membership probabilities on the Y-axis. The fill colour will indicate the original population assignments. Each facet represents the original population assignment for each sample:

In [ ]:
p <- ggplot(dapc.results, aes(x=Sample, y=Posterior_membership_probability, fill=Assigned_Pop))
p <- p + geom_bar(stat='identity') 
p <- p + scale_fill_manual(values = cols) 
p <- p + facet_grid(~Original_Pop, scales = "free")
p <- p + theme(axis.text.x = element_text(angle = 90, hjust = 1, size = 8))
p

This bar plot shows us a more organized perspective of our data set by contrasting the population membership probability assignments against their original populations.